In [27]:
import tensorflow as tf
import os
import mediapipe as mp
import pickle
import numpy as np
from tensorflow import keras
from keras import layers
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import keras_tuner
from keras_tuner.tuners import RandomSearch
from sklearn.utils import shuffle

Import the data using pickle

In [28]:
data_dict = pickle.load(open('./data1.pickle','rb'))
data = data_dict['data']
labels = data_dict['labels']
data, labels = shuffle(data, labels)

In [29]:
unwanted = []
for i in range(len(data)):
    if(len(data[i]) !=42):
        unwanted.append(i)
ele = unwanted[0]
del data[ele]
del labels[ele]
data = np.asarray(data)
labels = np.asarray(labels)

In [30]:
proper_label = {}
for i in range(37):
    proper_label[str(i)] = i

In [31]:
temp = []
for i in labels:
    temp.append(proper_label[i])
temp = np.array(temp)

Creating Neural network followed by data augmentation

In [32]:
def build_model(hp):
    model = keras.Sequential()
    
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units1, activation='relu',input_shape = (42,)))

    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units2, activation='relu'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    
    model.add(layers.Dense(37, activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [33]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=r'C:\Users\Anirudh Sai\Desktop\Project_P',
    project_name='ASL_Project')

In [34]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [35]:
tuner.search(data, temp,epochs = 20, validation_split = 0.2,batch_size = 30)

Trial 5 Complete [00h 01m 58s]
val_accuracy: 0.9718987743059794

Best val_accuracy So Far: 0.997205396493276
Total elapsed time: 00h 10m 59s
INFO:tensorflow:Oracle triggered exit


In [36]:
tuner.results_summary()

Results summary
Results in C:\Users\Anirudh Sai\Desktop\Project_P\ASL_Project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units1: 384
units2: 320
learning_rate: 0.001
Score: 0.997205396493276

Trial 3 summary
Hyperparameters:
units1: 288
units2: 448
learning_rate: 0.001
Score: 0.9965843756993612

Trial 0 summary
Hyperparameters:
units1: 384
units2: 32
learning_rate: 0.001
Score: 0.9936862786610922

Trial 4 summary
Hyperparameters:
units1: 128
units2: 160
learning_rate: 0.0001
Score: 0.9718987743059794

Trial 1 summary
Hyperparameters:
units1: 32
units2: 480
learning_rate: 0.0001
Score: 0.9655332962671915


In [37]:
model1 = keras.Sequential()
model1.add(keras.layers.Dense(units = 384, activation  = 'relu',input_shape = (42,) ))
model1.add(keras.layers.Dense(units = 320, activation  = 'relu'))
model1.add(layers.Dense(37, activation='softmax'))
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
model1.fit(data, temp,validation_split = 0.2,epochs=40,batch_size = 20)

Epoch 1/40
1289/1289 [==============================] - 4s 3ms/step - loss: 0.6961 - accuracy: 0.8038 - val_loss: 0.1682 - val_accuracy: 0.9677
Epoch 2/40
1289/1289 [==============================] - 3s 3ms/step - loss: 0.1438 - accuracy: 0.9605 - val_loss: 0.0910 - val_accuracy: 0.9803
Epoch 3/40
1289/1289 [==============================] - 4s 3ms/step - loss: 0.0930 - accuracy: 0.9735 - val_loss: 0.0748 - val_accuracy: 0.9786
Epoch 4/40
1289/1289 [==============================] - 3s 2ms/step - loss: 0.0719 - accuracy: 0.9800 - val_loss: 0.0456 - val_accuracy: 0.9885
Epoch 5/40
1289/1289 [==============================] - 4s 3ms/step - loss: 0.0555 - accuracy: 0.9847 - val_loss: 0.0516 - val_accuracy: 0.9868
Epoch 6/40
1289/1289 [==============================] - 4s 3ms/step - loss: 0.0514 - accuracy: 0.9851 - val_loss: 0.0285 - val_accuracy: 0.9927
Epoch 7/40
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0413 - accuracy: 0.9880 - val_loss: 0.0489 - val_accuracy:

In [26]:
model1.save('model1.h5')